In [ ]:
import numpy as np
import pandas as pd
import datetime

start_time = datetime.datetime.now()
train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
#print(train_identity['TransactionID'].size, train_identity['TransactionID'].unique().size)
#print(train_identity.loc[:, train_identity.isnull().any()])
    
#print(train_identity['id_17'].value_counts())
#for col in train_identity.loc[:, train_identity.isnull().any()].columns:
#    print(train_identity[col].value_counts())

In [ ]:
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
#print(train_transaction['TransactionID'].size, train_transaction['TransactionID'].unique().size)
#print(train_transaction)

In [ ]:
train = pd.concat([train_identity, train_transaction], axis=1)
#print(train)

In [ ]:
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

In [ ]:
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

In [ ]:
test = pd.concat([test_identity, test_transaction], axis=1)

*chunks*

In [ ]:
train_chunk = train#.sample(frac=0.05, random_state=1)
y_chunk = train_chunk['isFraud']
train_chunk.drop(['isFraud'], axis=1, inplace=True)
y_chunk.value_counts()

train = 0

Feature engineering

In [ ]:
nan_cols = set()

In [ ]:
for col in train_chunk.loc[:, train_chunk.isnull().any()].columns:
    #print(col)
    vc = train_chunk[col].value_counts(dropna=False, normalize=True)
    if vc.loc[np.nan] > 0.9:
        nan_cols.add(col)

#train_chunk['id_12'].value_counts(dropna=False, normalize=True)

In [ ]:
len(nan_cols)#, nan_cols

In [ ]:
for col in test.loc[:, test.isnull().any()].columns:
    #print(col)
    vc = test[col].value_counts(dropna=False, normalize=True)
    if vc.loc[np.nan] > 0.9:
        nan_cols.add(col)

In [ ]:
len(nan_cols)#, nan_cols

In [ ]:
one_value_cols = set()

In [ ]:
for col in train_chunk.columns:
    #print(col)
    vc = train_chunk[col].value_counts(normalize=True)
    if vc[vc.index[0]] > 0.9:
        #print(col)
        #print(vc.index[0], vc[vc.index[0]])
        one_value_cols.add(col)

In [ ]:
len(one_value_cols)#, one_value_cols

In [ ]:
for col in test.columns:
    #print(col)
    vc = test[col].value_counts(normalize=True)
    if vc[vc.index[0]] > 0.9:
        #print(col)
        #print(vc.index[0], vc[vc.index[0]])
        one_value_cols.add(col)

In [ ]:
len(one_value_cols)#, one_value_cols

In [ ]:
drop_this = list(nan_cols.union(one_value_cols))
len(drop_this)

*concatenate train+test*

In [ ]:
data = pd.concat([train_chunk, test], sort=False)
data.drop(drop_this, axis=1, inplace=True)
size = train_chunk.shape[0]
train_chunk = 0
test = 0

In [ ]:
def id30(x):
    if pd.isnull(x):
        return x    
    elif 'windows' in x.lower():
        return 'Windows'
    elif 'ios' in x.lower():
        return 'iOS'
    elif 'mac' in x.lower():
        return 'Mac OS'
    elif 'android' in x.lower():
        return 'Android'
    else:
        return x
    
data['id_30'] = data['id_30'].apply(id30)
data['id_30'].value_counts()

In [ ]:
def id31(x):    
    if pd.isnull(x):
        return x
    elif 'chrome' in x.lower():
        return 'Chrome' 
    elif 'safari' in x.lower():
        return 'Safari'
    elif 'firefox' in x.lower():
        return 'Firefox'
    elif 'ie' in x.lower():
        return 'Internet Explorer'
    elif 'edge' in x.lower():
        return 'Microsoft Edge'
    elif 'samsung' in x.lower():
        return 'Samsung Browser'
    elif 'google' in x.lower():
        return 'Google Search Application'
    elif 'opera' in x.lower():
        return 'Opera'
    elif 'android' in x.lower():
        return 'Android Browser'
    elif 'facebook' in x.lower():
        return 'Facebook'    
    else:
        return 'other'

data['id_31'] = data['id_31'].apply(id31)
data['id_31'].value_counts()

In [ ]:
data['id_33_hor'] = data['id_33'].apply(lambda x: x if pd.isnull(x) else x.split('x')[0])
data['id_33_vert'] = data['id_33'].apply(lambda x: x if pd.isnull(x) else x.split('x')[1])
data.drop(['id_33'], axis=1, inplace=True)

In [ ]:
def devinfo(x):
    if pd.isnull(x):
        return x    
    elif 'rv:' in x.lower():
        return 'rv'
    elif 'sm-' in x.lower():
        return 'SM'
    elif 'moto' in x.lower():
        return 'Moto'
    elif ('huawei' in x.lower()) or ('-l' in x.lower()):
        return 'HUAWEI'
    elif 'lg-' in x.lower():
        return 'LG'
    elif 'pixel' in x.lower():
        return 'Pixel'
    elif 'samsung' in x.lower():
        return 'Samsung'
    elif 'windows' in x.lower():
        return 'Windows'
    elif ('ios device' in x.lower()) or ('macos' in x.lower()) or ('trident' in x.lower()):
        return x
    else:
        return 'other'

data['DeviceInfo'] = data['DeviceInfo'].apply(devinfo)
data['DeviceInfo'].value_counts()

In [ ]:
def p_email(x):
    if pd.isnull(x):
        return x    
    elif 'gmail' in x:
        return 'gmail'
    if 'yahoo' in x:
        return 'yahoo'
    if 'hotmail' in x:
        return 'hotmail'
    if 'outlook' in x:
        return 'outlook'
    if 'anonymous' in x:
        return 'anonymous'
    if 'aol' in x:
        return 'aol'
    if 'comcast' in x:
        return 'comcast'
    if 'icloud' in x:
        return 'icloud'
    if 'live' in x:
        return 'live'
    else:
        return 'other'
    
data['P_emaildomain'] = data['P_emaildomain'].apply(p_email)
data['P_emaildomain'].value_counts()

In [ ]:
data['R_emaildomain'] = data['R_emaildomain'].apply(p_email)
data['R_emaildomain'].value_counts()

**Missing Values**

In [ ]:
def missing_values(df):
    for label, col in df.loc[:, df.isnull().any()].iteritems():    
        missing = df[label].isnull()
        vc = col.value_counts(normalize=True)    
        df.loc[missing, label] = np.random.choice(vc.index, size=col.size-col.count(), p=vc.values)    

In [ ]:
#print(train['id_01'].value_counts())
#train_copy = train
#missing_values(train_copy)

In [ ]:
#test_copy = test
#missing_values(test_copy)

In [ ]:
missing_values(data)

*dummies*

In [ ]:
data = pd.get_dummies(data)

In [ ]:
data.shape

*scaling*

In [ ]:
from sklearn.preprocessing import StandardScaler, scale

data = scale(data)

scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_final)

In [ ]:
print(train_final.shape)
print(test_final.shape)
scaled_test = scaler.transform(test_final)

In [ ]:
train_final = data[:size]
test_final = data[size:]

In [ ]:
#train_final.to_csv('train_final.csv')

In [ ]:
#test_final.to_csv('test_final.csv')

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

**LogisticRegression**

In [ ]:
for c in np.power(10.0, np.arange(-5, 6)):
    print('C={}'.format(c))
    print('Score {}\n'.format(cross_val_score(LogisticRegression(C=c, solver='lbfgs', max_iter=300), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()))
    
#best_logreg = 'C=0.001, Score 0.792172393105506'

**SVC**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

best_svc = [0, 0, 0]
for c in np.power(10.0, np.arange(-5, 6)):
    for g in np.power(10.0, np.arange(-5, 6)):
        print('C={}, gamma={}'.format(c, g))
        svc_score = cross_val_score(SVC(C=c, gamma=g), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()
        print('Score {}\n'.format(svc_score))
        if svc_score > best_svc[2]:
            best_svc = [c, g, svc_score]
            
print('C={}, gamma={}, Score {}'.format(best_svc[0], best_svc[1], best_svc[2]))
#best_svc = 'C=0.1, gamma=0.001, Score 0.7483778558809864'

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

rf_score = []
n = [50, 100, 150, 200]
for k in n:
    print('K={}'.format(k))
    rf_score.append(cross_val_score(RandomForestClassifier(n_estimators=k, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())
    print('Score {}\n'.format(rf_score[-1]))
    
estim = n[np.argmax(rf_score)] #0.8456387249804596

**Gradient Boosting**

In [ ]:

gb_score = []
n = [100, 125, 150, 175, 200]
lrate = [0.1, 0.05, 0.01]
for k in n:
    for l in lrate:
        print('K={}, Learning rate = {}'.format(k, l))
        gb_score.append(cross_val_score(GradientBoostingClassifier(n_estimators=k, learning_rate=l, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())
        print('Score {}\n'.format(gb_score[-1]))
        
#best_gb = 'K=125, Learning rate = 0.1, Score 0.8395876357222953'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,10))
sns.lineplot(n, rf_score)
#sns.lineplot(lrate, gb_score[:3], label='100')
#sns.lineplot(lrate, gb_score[3:6], label='125')
#sns.lineplot(lrate, gb_score[6:9], label='150')
#sns.lineplot(lrate, gb_score[9:12], label='175')
#sns.lineplot(lrate, gb_score[12:], label='200')

Prediction

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#cls = RandomForestClassifier(n_estimators=estim, random_state=1)
cls = GradientBoostingClassifier(n_estimators=200, random_state=1)
cls.fit(train_final, y_chunk)

In [ ]:
submission['isFraud'] = cls.predict_proba(test_final)[:, 1]
#submission['isFraud']
submission.to_csv('submission.csv')